# Initial Setup
- Initializing and authenticating google earth engine
- Setting up folium

In [5]:
import ee
import geemap
import os
import numpy as np
import branca
import folium

# Authenticate to Earth Engine
try:
  ee.Initialize(project='envr451-2024')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='envr451-2024')

In [ ]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


# Planet
Planet provides us with 4.77m resolution all the way to Jan 2016.
https://developers.google.com/earth-engine/datasets/catalog/projects_planet-nicfi_assets_basemaps_americas

Note - with the NICFI project, we can only get biannual images up until 2020; in 2020 we get
planet_medres_normalized_analytic_2020-06_2020-08_mosaic
planet_medres_normalized_analytic_2020-09_mosaic
planet_medres_normalized_analytic_2020-10_mosaic
planet_medres_normalized_analytic_2020-11_mosaic
planet_medres_normalized_analytic_2020-12_mo

Here, I am considering medians for all years. Consider, however, that some images have clouds, and we expect worse quality for the images from 2015-2020 since 2015-2019 only has 2 images per year, and 2020 has 5, while for 2021-2024 we get all 12 images for the year.

This can be improved once we get all images from McGill. Also, these should be at 3.7m resolution.saic

In [488]:
piura = ee.FeatureCollection(ee.FeatureCollection('projects/envr451-2024/assets/piura').geometry().bounds())
# task = ee.batch.Export.table.toDrive(
#     collection=piura,
#     description = 'piura_region',
#     fileNamePrefix = 'piura_region',
#     fileFormat = 'kmz'
# )
# task.start()
basemap = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas').filter(ee.Filter.date('2015-01-01', '2024-12-31')) \
            .map(lambda image: image.clip(piura))

# Define the years for which you want to calculate geometric medians.
years = ee.List.sequence(2015, 2024)

# Function to calculate geometric median for each year.
def calculate_median(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    yearly_collection = basemap.filterDate(start_date, end_date)
    median = yearly_collection.reduce(ee.Reducer.median())
    return median.set('year', year)

# Map over the list of years and calculate geometric medians.
yearly_medians = ee.ImageCollection(years.map(calculate_median)).map(lambda image: image.clip(piura))
yearly_medians_list = yearly_medians.toList(yearly_medians.size())


In [ ]:
vis_basemap = {'bands': ['R_median', 'G_median', 'B_median'], 'min': 64, 'max': 5454, 'gamma': 1.8}

# # Get the number of images.
# num_images = yearly_medians_list.size().getInfo()

# # Loop over the list and export each image.
# for i in range(num_images):
image = ee.Image(yearly_medians_list.get(1))

m = folium.Map(location = [8.5, -81], tiles="openstreetmap", zoom_start=13.4,
               zoom_control=False, scrollWheelZoom=False, dragging=False)
f = folium.Figure(width=1000, height=1000)
f.add_child(m)

# Add custom basemaps
basemaps['Esri Satellite'].add_to(m)

# Add the elevation model to the map object.
m.add_ee_layer(image, vis_basemap)

# Define the latitude and longitude intervals
lat_interval = 0.01
lon_interval = 0.01

# Create the grid
for lat in np.arange(8, 9, lat_interval):
    for lon in np.arange(-82, -80, lon_interval):
        # Convert numpy.float64 to float
        lat, lon = float(lat), float(lon)
        # Create a box for each grid cell
        grid_cell = folium.Polygon(
            locations=[(lat, lon), (lat, lon + lon_interval), 
                       (lat + lat_interval, lon + lon_interval), (lat + lat_interval, lon)],
            color="#000000",  # Line color
            fill=False,  # Defines if the polygon is filled
            weight=1,  # Line weight
        )
        # Add the grid cell to the map
        m.add_child(grid_cell)
        
# Get the number of images.
# num_images = yearly_medians_list.size().getInfo()

# # Loop over the list and export each image.
# for i in range(num_images):
#     year = years.get(i).getInfo()
#     task = ee.batch.Export.image.toDrive(
#         image=ee.Image(yearly_medians_list.get(i)),
#         description = 'Planet_Satellite_5m_median_' + str(year),
#         fileNamePrefix = 'Planet_Satellite_5m_median_' + str(year),
#         region=piura,
#         scale=5,
#         maxPixels=1e13
#     )
#     task.start()

# Map = geemap.Map()
# Map.centerObject(piura, 13)
# Map.addLayer(yearly_medians.first(), vis_basemap, 'last_image')
# Map

# Copernicus Land Cover
100m resolution
2015-01-01 to 2019-12-31

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#description

In [454]:
dataset = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select('discrete_classification').clip(piura)

palette = [
  '006400', # forest (10)
  'ffbb22', # shrubland (20)
  'ffff4c', # grassland (30)
  'f096ff', # cropland (40)
  'fa0000', # urban (50)
  'b4b4b4', # bare/sparse (60)
  'f0f0f0', # snow (70)
  '0064c8' # water (80)
]

visualization = {
    'min':10, 'max': 80, 'palette': palette,
}
grid = geemap.latlon_grid(
    lat_step=0.01, lon_step=0.01, west=-81.25, east=-80.75, south=8.25, north=9.25
)

m = geemap.Map()
m.center_object(dataset1)
m.add_layer(dataset1, visualization, 'Landcover')
m.add_layer(dataset2, visualization, 'Landcover2')
m.addLayer(grid, {}, "Coordinate Grid")
# m

# ESA WorldCover
10m resolution
2020-01-01 - 2022-12-31

https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100
https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200

In [330]:
dataset1 = ee.ImageCollection('ESA/WorldCover/v100').map(lambda image: image.clip(piura))
dataset2 = ee.ImageCollection('ESA/WorldCover/v200').map(lambda image: image.clip(piura))

palette = [
  '006400', # forest (10)
  'ffbb22', # shrubland (20)
  'ffff4c', # grassland (30)
  'f096ff', # cropland (40)
  'fa0000', # urban (50)
  'b4b4b4', # bare/sparse (60)
  'f0f0f0', # snow (70)
  '0064c8' # water (80)
]

visualization = {
    'min':10, 'max': 80, 'palette': palette,
}

# m = geemap.Map()
# m.center_object(dataset1)
# m.add_layer(dataset1, visualization, 'Landcover')
# m.add_layer(dataset2, visualization, 'Landcover2')
# m

# ESRI Land Use Land Cover

https://livingatlas.arcgis.com/landcover/

In [456]:
esri_lulc2020 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m").mosaic().clip(piura)

dict = {
  "names": [
    "Water", "Trees", "Grass","Flooded Vegetation","Crops","Scrub/Shrub",
    "Built Area","Bare Ground","Snow/Ice","Clouds"
  ],
  "colors": [
    "1A5BAB","358221","A7D282","87D19E","FFDB5C","EECFA8",
    "ED022A","EDE9E4","F2FAFF","C8C8C8"
  ]
}

# Create a palette from the dict object
palette = dict['colors']

# Create a visualization parameter object
visualization = {'min': 1, 'max': len(dict['names']), 'palette': palette}


# NASA Elevation and Sentinel Radar

In [501]:
# Import the dataset and select the elevation band.
dataset = ee.Image('NASA/NASADEM_HGT/001')
elevation = dataset.select('elevation')

# Set elevation visualization properties.
elevation_vis = {'min': 0, 'max': 2500, 'palette': ['green', 'red']}

# Set elevation <= 0 as transparent.
elevation_masked = elevation.updateMask(elevation.gt(0)).clip(piura)

In [499]:
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)

img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

# desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

# desc_change = ee.Image.cat(
#     desc.filter(spring).mean(),
#     desc.filter(late_spring).mean(),
#     desc.filter(summer).mean(),
# )

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
).clip(piura)

m = geemap.Map()
m.centerObject(piura)
m.add_layer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
# m.add_layer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
m.addLayer(elevation_masked, elevation_vis, 'Elevation')
# m.add_layer(piura, {})
m


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [498]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003').clip(piura)

# elev = dem.sample(xy, 30).first().get('elevation').getInfo()
# print('Mount Everest elevation (m):', elev)

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
# my_map = folium.Map()

m = folium.Map(location = [8.5, -81], tiles="openstreetmap", zoom_start=13.4,
               zoom_control=False, scrollWheelZoom=False, dragging=False)
f = folium.Figure(width=1000, height=1000)
f.add_child(m)

# Add custom basemaps
basemaps['Esri Satellite'].add_to(m)

# Add the elevation model to the map object.
m.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Define the latitude and longitude intervals
lat_interval = 0.01  # or any decimal value you want
lon_interval = 0.01  # or any decimal value you want

# Create the grid
for lat in np.arange(8, 9, lat_interval):
    for lon in np.arange(-82, -80, lon_interval):
        # Convert numpy.float64 to float
        lat, lon = float(lat), float(lon)
        # Create a box for each grid cell
        grid_cell = folium.Polygon(
            locations=[(lat, lon), (lat, lon + lon_interval), 
                       (lat + lat_interval, lon + lon_interval), (lat + lat_interval, lon)],
            color="#000000",  # Line color
            fill=False,  # Defines if the polygon is filled
            weight=1,  # Line weight
        )
        # Add the grid cell to the map
        m.add_child(grid_cell)

# Define the legend HTML content
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; z-index:9999; background-color: white; padding: 10px; border: 1px solid black; border-radius: 5px;">
    <b>Elevation Legend</b><br>
    <div style="background-color: #006633; width: 20px; height: 20px; display: inline-block;"></div> 0 - 1000 m <br>
    <div style="background-color: #E5FFCC; width: 20px; height: 20px; display: inline-block;"></div> 1000 - 2000 m <br>
    <div style="background-color: #662A00; width: 20px; height: 20px; display: inline-block;"></div> 2000 - 3000 m <br>
    <div style="background-color: #D8D8D8; width: 20px; height: 20px; display: inline-block;"></div> 3000 - 4000 m <br>
    <div style="background-color: #F5F5F5; width: 20px; height: 20px; display: inline-block;"></div> No Data <br>
</div>
'''

# Add the legend HTML content to the map as a control
m.get_root().html.add_child(folium.Element(legend_html))

# Add a layer control panel to the map
m.add_child(folium.LayerControl())

m.save('map.html')

hti = Html2Image(size=(10000, 10000), browser='edge')
hti.screenshot(html_file='map.html', save_as='elevation.png')



['C:\\Users\\anaca\\Desktop\\piura\\elevation.png']

In [375]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
    
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-12-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)
dataset

# visualization = {
#     'min': 0.0,
#     'max': 0.3,
#     'bands': ['B4', 'B3', 'B2'],
# }

# m = geemap.Map()
# m.centerObject(piura)
# m.add_layer(dataset.median(), visualization, 'RGB')
# m

In [389]:
dataset = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")

treeloss_year = dataset.select(['lossyear']).clip(piura)
treeLossVisParam = {'min': 0, 'max': 22, 'palette': ['yellow', 'red']}

# m = geemap.Map()
# m.add_basemap('Esri.WorldImagery')
# m.center_object(piura)
# m.add_layer(treeloss_year, treeLossVisParam, 'Tree loss year')
# m.add_colorbar(treeLossVisParam, label=layer_name, layer_name='Tree loss year')
# m.add('layer_manager')
# m

# task = ee.batch.Export.image.toDrive(
#     image=treeloss_year,
#     description = 'treeloss_year_2001_2022_30m',
#     fileNamePrefix = 'treeloss_year_2001_2022_30m',
#     region=piura,
#     scale=30,
#     maxPixels=1e13
# )

# task.start()